In [ ]:
from PIL import Image
import os
import random
import math
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

In [ ]:
base_path = "./roboflow_datasets/cats-13/train"
destination = "./cropped_roboflow4-224"
crop_size = 224
train = 0.7
val = 0.15
test = 0.15

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip with 50% probability
    iaa.Flipud(0.5),  # Vertical flip with 50% probability
    iaa.OneOf([  # Either contrast normalization or color augmentation
        iaa.ContrastNormalization((0.75, 1.25)),
        iaa.Multiply((0.8, 1.2)),
        iaa.Grayscale(alpha=(0.0, 0.1))
    ]),
    iaa.OneOf([  # Either blur or noise transformation
        iaa.GaussianBlur(sigma=(0.0, 1)),
        iaa.AverageBlur(k=(2, 3)),
        iaa.MedianBlur(k=(3)),
        iaa.BilateralBlur(d=(3, 5), sigma_color=(10, 50), sigma_space=(10, 50)),
        iaa.AdditiveGaussianNoise(scale=(0.0, 0.01 * 255)),
        iaa.ElasticTransformation(alpha=2, sigma=1)
    ])
])

rotate = iaa.Affine(rotate=(-45, 45))

In [ ]:
def get_keypoints_from_line(line, height, width):
    data = line.strip().split()
    
    coords = list(map(float, data[1:]))
    
    points = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
    
    keypoints = [Keypoint(x=x*width, y=y*height) for x, y in points]

    return keypoints
def augment_image_and_keypoints(image, line, sequence):
    image_np = np.array(image, dtype=np.uint8)

    height, width = image_np.shape[:2]
    keypointsOnImage = KeypointsOnImage(get_keypoints_from_line(line, height, width), shape=image_np.shape)
    
    image_aug_np, keypoints_aug = sequence(image=image_np, keypoints=keypointsOnImage)
    
    image_aug_pil = Image.fromarray(image_aug_np)
    
    return image_aug_pil, keypoints_aug

In [ ]:
image_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".jpg"):
            image_paths.append(os.path.join(root, file))

In [ ]:
mask_paths = []
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".txt"):
            mask_paths.append(os.path.join(root, file))

In [ ]:
for image_path in image_paths:
    for _ in range(5):
        image = Image.open(image_path)
        mask_path = image_path.replace("jpg", "txt")
        with open(mask_path, "r") as mask_file:
            lines = mask_file.readlines()
            

In [ ]:
crops = []
for image_path in image_paths:
    for _ in range(5):
        image = Image.open(image_path)
        width, height = image.size
        #mask_path = os.path.join(base_path, image_path.split("/")[2], "labels", os.path.basename(image_path).replace(".jpg", ".txt"))
        mask_path = os.path.join(base_path, "labels", os.path.basename(image_path).replace(".jpg", ".txt"))
        with open(mask_path, "r") as f:
            lines = f.readlines()
            empty = False
            if len(lines) == 0:
                empty = True
                x_min_valid = 0
                x_max_valid = width - crop_size
                y_min_valid = 0
                y_max_valid = height - crop_size
            else:
                line = lines[0]
                coordinates = line.split(" ")[1:]
                relative_x = [float(x) * width for x in coordinates[::2]]
                relative_y = [float(y) * height for y in coordinates[1::2]]

                x_min = min(relative_x)
                x_max = max(relative_x)
                y_min = min(relative_y)
                y_max = max(relative_y)

                x_min_valid = max(0, x_max - crop_size)
                x_max_valid = min(width - crop_size, x_min)
                y_min_valid = max(0, y_max - crop_size)
                y_max_valid = min(height - crop_size, y_min)

                if x_min_valid >= x_max_valid or y_min_valid >= y_max_valid or x_max_valid - x_min_valid < 0.5 or y_max_valid - y_min_valid < 0.5:
                    print("Invalid crop for image " + image_path)
                    continue

            crop_x = random.randint(math.ceil(x_min_valid), math.floor(x_max_valid))
            crop_y = random.randint(math.ceil(y_min_valid), math.floor(y_max_valid))

            crop = image.crop((crop_x, crop_y, crop_x + crop_size, crop_y + crop_size))
            
            if not empty:
                relative_x = [(x - crop_x) for x in relative_x]
                relative_y = [(y - crop_y) for y in relative_y]
                
                string = '0 '
                for x, y in zip(relative_x, relative_y):
                    string += str(x / crop_size) + ' ' + str(y / crop_size) + ' '
            else:
                string = ''
            crops.append((crop, string))

In [ ]:
for i in range(len(crops)):
    image, line = crops[i]
    try:
        image, keypoints = augment_image_and_keypoints(image, line, seq)
    except:
        continue
    line = ''
    if len(keypoints.keypoints) > 0:
        line = '0 '
    for keypoint in keypoints.keypoints:
        if keypoint.x < 0 or keypoint.y < 0 or keypoint.x > crop_size or keypoint.y > crop_size:
            print("Invalid keypoint: " + str(keypoint.x/crop_size) + " " + str(keypoint.y/crop_size))
        line += str(keypoint.x/crop_size) + ' ' + str(keypoint.y/crop_size) + ' '
    crops[i] = (image, line)

In [ ]:
train_size = int(len(crops) * train)
val_size = int(len(crops) * val)
test_size = int(len(crops) * test)

In [ ]:
train_indices = random.sample(range(len(crops)), train_size)

remaining_indices = [i for i in range(len(crops)) if i not in train_indices]
val_indices = random.sample(remaining_indices, val_size)

test_indices = [i for i in range(len(crops)) if i not in train_indices and i not in val_indices]

train_crops = [crops[i] for i in train_indices]
val_crops = [crops[i] for i in val_indices]
test_crops = [crops[i] for i in test_indices]

In [ ]:
print(len(image_paths))
print(len(crops))
print(len(train_crops))
print(len(val_crops))
print(len(test_crops))

In [ ]:
folder = "train"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in train_crops:
    name = str(random.randint(0, 1000000))
    #image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    # with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        # f.write(data)
    values = data.split()
    values = [float(x) for x in values]
    if any([x < 0 or x > 1 for x in values]):
        print(values)

In [ ]:
folder = "test"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in test_crops:
    name = str(random.randint(0, 1000000))
    image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        f.write(data)

In [ ]:
folder = "valid"
os.makedirs(os.path.join(destination, folder, "images"), exist_ok=True)
os.makedirs(os.path.join(destination, folder, "labels"), exist_ok=True)
for image, data in val_crops:
    name = str(random.randint(0, 1000000))
    image.save(os.path.join(destination, folder, "images", name + ".jpg"))
    with open(os.path.join(destination, folder, "labels", name + ".txt"), "w") as f:
        f.write(data)